In [ ]:
!pip install torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import pipeline
import math

# Long-Context Summarization using BigBirdPegasus for long documents.
long_doc_summarizer = pipeline(
    "summarization",
    model="google/bigbird-pegasus-large-arxiv",
    framework="pt"
)

# A sample long financial document (get lines from online to test this).
long_financial_text = """
Morgan Stanley reported a 10% increase in revenue last quarter, driven by strong performance in its wealth management division.
The bank also noted that its investment banking group contributed significantly to overall growth despite a challenging market environment.
Additionally, cost reductions and process improvements helped improve profit margins.

JPMorgan Chase, on the other hand, announced that it will cut operational costs by 5% over the next fiscal year.
The bank believes that these savings will bolster its long-term profitability, although analysts remain cautious about the execution of such large-scale cost-cutting measures.
The company’s CEO mentioned that these steps are part of a broader strategic initiative to adapt to a rapidly changing economic landscape.

Apple Inc. recently unveiled its latest iPhone model at a high-profile event in Cupertino.
The company expects that the new device will capture a significant share of the smartphone market, particularly in emerging markets where demand is growing rapidly.
The product launch was accompanied by robust pre-orders and positive early reviews, reinforcing the company’s market-leading position.

Goldman Sachs recorded record earnings in Q1 2024, attributing its success to robust investment strategies and aggressive cost management.
The bank’s strategic bets in the technology sector and selective risk-taking have paid off, although market volatility remains a concern for investors.
Analysts have noted that while short-term performance is strong, sustaining this growth will require continuous innovation and prudent risk management.

Wells Fargo announced that it will raise its dividend next quarter.
The bank stated that it has sufficient capital reserves to support the increase, even as it navigates a challenging regulatory environment.
Investors view this move as a positive signal of the bank’s confidence in its long-term financial health.
"""


Device set to use cuda:0


In [ ]:
# Direct long-context summarization. Usually this looks good
print("Long-Context Summarization ")
summary_direct = long_doc_summarizer(long_financial_text, max_length=200, min_length=100, do_sample=False)
print(summary_direct[0]['summary_text'])
print( "=" * 80 + "\n")




Attention type 'block_sparse' is not possible if sequence_length: 336 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


=== Direct Long-Context Summarization ===
selective risktaking remains prudent and market volatility will remain concern for short-term performance is strong , sustaining this growth while prudent and prudent risk management noted that it will raise its capital will pay off in next fiscal quarter . recently unveiled its latest cautious model at high-profile event in a rapidly changing strategic initiative in company expects that new device will capture a significant share of the market in emerging markets . recently announced its latest cautious model at high-profile event in a rapidly changing strategic initiative in company expects that new device will capture a significant share of the market in emerging markets . <n> recently announced its latest cautious model at high-profile event in a rapidly changing strategic initiative in company expects that new device will capture a significant share of the market in emerging markets . <n> recently announced its latest cautious model at hig

In [ ]:
#Hierarchical Summarization
# 1. split the long document into segments.

def split_text(text, num_segments):

    # Try splitting by newline.
    lines = text.strip().split("\n")
    if len(lines) >= num_segments:
        segment_size = math.ceil(len(lines) / num_segments)
        segments = [' '.join(lines[i*segment_size:(i+1)*segment_size]) for i in range(num_segments)]
    else:
        words = text.split()
        segment_size = math.ceil(len(words) / num_segments)
        segments = [' '.join(words[i*segment_size:(i+1)*segment_size]) for i in range(num_segments)]
    return segments

# 2. Split the document into 3 segments
segments = split_text(long_financial_text, num_segments=3)





In [ ]:
# Summarize each segment individually.
segment_summaries = []
print("=== Hierarchical Summarization: Segment Summaries ===")
for idx, segment in enumerate(segments, start=1):
    seg_sum = long_doc_summarizer(segment, max_length=120, min_length=60, do_sample=False)
    segment_summary = seg_sum[0]['summary_text']
    segment_summaries.append(segment_summary)
    print(f"Segment {idx} summary:")
    print(segment_summary)


# Combine the segment summaries to see
combined_summary_text = " ".join(segment_summaries)

print(combined_summary_text)
print( "=" * 80 + "\n")

=== Hierarchical Summarization: Segment Summaries ===
Segment 1 summary:
company reported strong performance in its wealth division driven by strong performance in its investment banking group and driven by strong performance in its wealth management division .<n> the bank also reported strong performance in its operational costs .<n> the bank stated that its long-term costs will be reduced by a factor of @xmath0 over the next five years . <n> company reported strong performance in its wealth division driven by strong performance in its investment banking group and driven by strong performance in its wealth management division .<n> the bank also stated that its long-term costs will be reduced by a factor of
----------------------------------------


Your max_length is set to 120, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)


Segment 2 summary:
recent years have seen a rapid rise in the demand for mobile computing devices such as smartphones and tablet computers .<n> however , in spite of this rapid growth , the cost of making such devices has remained relatively high .<n> recently , a new company has emerged in the field of nanoelectronics .<n> this company , named as  bncore '' , aims to bring down the cost of making a single nanoelectronic device to a level comparable with the cost of making a single molecule of silicon .<n> bncore s nanoelectronics division is
----------------------------------------
Segment 3 summary:
in this note we present the results of the first quarter of 2011 .<n> we report results for the fully integrated : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : :
----------------------------------------

Combined Segment Summaries:
company reported strong performance in its wealth division driven by strong performance in its investment bankin

In [ ]:
# Summarize the combined summary text
overall_summary = long_doc_summarizer(combined_summary_text, max_length=200, min_length=100, do_sample=False)
print("=== Overall Hierarchical Summary ===")
print(overall_summary[0]['summary_text'])


=== Overall Hierarchical Summary ===
recently , a new company has emerged in the field of nanoelectronics , named as bncore '' , aims to bring down the cost of making a single nanoelectronic device to a level comparable with the cost of making a single molecule of silicon .<n> bncore s nanoelectronics division is in this note we present the results of the first quarter of 2011 . <n> we report results for the fully integrated : : : : : : : : : : : bncore s nanoelectronics division is in this note we present the results of the first quarter of 2011 .
